# **RNAseq Analysis Module**

## **Practical session 4: Mapping output quality check**

Wednesday, the 2nd of December, 2020   
Claire Vandiedonck and Sandrine Caburet - 2020   


   1. Quality reports with qualimap, on sorted bam files (outputs of mapping with bowtie)  
   2. multiqc to Generation of a single report for all quality checks at once with multiqc  
   3. Visualisation of the mapped data, with IGV 



---
## **Before going further**

<div class="alert alert-block alert-danger"><b>Caution:</b> 
Before starting the analysis, save a backup copy of this notebok : in the left-hand panel, right-click on this file and select "Duplicate"<br>
You can also make backups during the analysis. Don't forget to save your notebook regularly.
<div>

___

## **I - Mapping quality check**


After the mapping steps that we performed yesterday, we are going to check the outputs for the quality of the mapping procedure.

### **1. with QUALIMAP**
    
So first we'll run the **QUALIMAP** program (http://qualimap.conesalab.org/), that collects the data about the `.bam` file, including coverage estimation and many other parameters, and reports a summary of the main properties of the alignment data. QUALIMAP reads `.sorted.bam` files and generates a folder containing a report on `.html` format.

As you can see from the next command to know which version we are running, QUALIMAP includes several tools. We will be using the classical **bamqc** tool which can work on any kind of NGS `.bam` files. Of note, there is also a **rnaqc** tool which is dedicated to RNASeq data but it fails to run with C. Parapsilosis annotations.

In [ ]:
qualimap --help

> Remark: QUALIMAP can also be launched on most OS using a command line that will open a user-friendly Java window. On Mac or Windows, you may have to modify the memory RAMs to run it. It is all explained in the documentation.

In [ ]:
# Contents of the Results folder
ls -lh ./Results

The files analysed by QUALIMAP are the sorted bam files. Here are the ones correponding to the samples that were mapped yesterday: 

In [ ]:
ls -lh ./Results/*.sorted.bam

If we wanted to run QUALIMAP on a single file, e.g the Normoxia one, the command line woould be as follow:

To get a properly-named folder for each bam file analysed by Qualimap, the program will be run in a `for loop`, that will run the program once for each element in the provided list.

Let's unsdetsrand the command of the loop stepwise.

- First we list all the .bam files with `ls`

In [ ]:
ls ./Results/*.sorted.bam

- Then we identify the filenames and do a loop:

In [ ]:
for fn in $(ls ./Results/*.sorted.bam); do
    echo ${fn}
done

- Then we create a `mysortedbam` variable with the name of the file without the path. To do so, we use the command `basename`. 

In [ ]:
for fn in $(ls ./Results/*.sorted.bam); do
     
 mysortedbam=$(basename ${fn})
 echo ${mysortedbam}
 
done

- Then we generate an `id` with the prefix part of the fullid we want to keep. The command below allows to substitute what is between the `\` by nothing.

In [ ]:
for fn in $(ls ./Results/*.sorted.bam); do
     
    mysortedbam=$(basename ${fn})
    id=${mysortedbam/_bowtie_mapping.sorted.bam/}
    echo "========Processing sampleID: ${id}"
 
done

- Now we create a new variable `myoutdir` to specify the name of the output directory

In [ ]:
#Runs for multiple html outputs, with relevant names

for fn in $(ls ./Results/*.sorted.bam); do
     
    mysortedbam=$(basename ${fn})
    id=${mysortedbam/_bowtie_mapping.sorted.bam/}
    echo "========Processing sampleID: ${id}"
     
    myoutdir="./Results/${id}_qualimap" 
    echo ${myoutdir}
done

- Now we are ready to add the qualimap command!

In [ ]:
#Runs for multiple html outputs, with relevant names

for fn in $(ls ./Results/*.sorted.bam); do
     
    mysortedbam=$(basename ${fn})
    id=${mysortedbam/_bowtie_mapping.sorted.bam/}
    echo "========Processing sampleID: ${id}"
     
    myoutdir="./Results/${id}_qualimap" 
    echo ${myoutdir} 
    qualimap bamqc -bam ${fn} -gff /srv/data/meg-m2-rnaseq/genome/C_parapsilosis_ORFs.gff -outdir $myoutdir 
    echo "...done"

done

___

### **2. with SAMTOOLS**
    
**SAMTOOLS** we already used yesterday (v1.11) includes also very usefull tools to QC bams:



In [ ]:
mkdir ./Results/Flags_Stats

for fn in $(ls ./Results/*.sorted.bam); do

    mysortedbam=$(basename ${fn})
    id=${mysortedbam/_bowtie_mapping.sorted.bam/}
    echo "========Processing sampleID: ${id}"

    samtools stats ${fn} > ./Results/Flags_Stats/${id}.stats
    samtools flagstats ${fn} > ./Results/Flags_Stats/${id}.flagstats
done

You can open the generated files directly in JupyterLab by clicking on them from th eleft-hand panel.

___

## **II - Compiling the quality check reports**

When numerous samples are processed, it can easily become tedious to look in each mapping quality report. So we'll run **MultiQC** (https://multiqc.info/), that scans automatically a folder for all quality checks outputs and produce a single report. MULTIQC runs on almost any possible NGS tools (https://multiqc.info/docs/#multiqc-modules).


In [ ]:
multiqc --version

In [ ]:
# Here we do not specify modules for multiQC. 
multiqc --interactive ./Results/ -o ./Results/MultiQC

#By default, multiQC identifies any report it can parse from the input directory.
# If you want to only generate a multiQC report on specific analyses, you can add the argument -m followed by the name of the module
# example:
# multiqc -m fastqc ./Results/Fastqc/ -o/Results/MultiQC_on_FastQC
# you can add several modules -m fastqc dir_fastqc -m qualimap dirqualimap etc...

To open the report, double-click on the multiqc_report.html in the left-hand panel. Enjoy !

<div class="alert alert-block alert-success"><b>=> Question: What can you say on the data?</b><br>

*(you can click here to add your answers directly in this markdown cell)*<br>
    
- What are the proportions of reads properly mapped to the reference genome?
- What do you think about the depth?
- How do you explain that some reads were not mapped?
- Is there a link between the fastqc read qualities and the bam qualities?

___

## **III - Visualisation of mapped data with IGV**

IGV (Integrated Genome Viewer) (http://software.broadinstitute.org/software/igv/) is the most populat browser for visualising NGS data.

In Unix, the command to start IGV is `igv` or `igv.sh` depending on the name of the launching script. A Java window would thus open. 

Unfortunately, the widget for "mounting" IGV within a jupyter notebook is not ready yet (development in progress).

So we will look at the mapped data using the IGV desktop app (avalaible here: http://software.broadinstitute.org/software/igv/download)
To avoid any lengthy download, the data can be retrieved from the Galaxy history (cf Practical Session 2, https://usegalaxy.eu/) and pushed into IGV via the _*display with IGV local*_ link, available when "opening" the title of a dataset.  

> If you have trouble with your own history, one is available with relevant datasets here: https://usegalaxy.eu/u/scaburet/h/rnaseq-candida-data-analysis

<div class="alert alert-block alert-success"><b>=> Question: What can you say on the data?</b><br>

*(you can click here to add your answers directly in this markdown cell)*<br>
    
-  Is your RNASeq strand specific?
-  Do you see any difference between hypoxia and normoxia files?
-  Do your data fit the results of the paper? </div>

<div class="alert alert-block alert-success"><b>Success:</b> Don't forget to save you notebook and export a copy as an <b>html</b> file as well <br>
- Open "File" in the Menu<br>
- Select "Export Notebook As"<br>
- Export notebook as HTML<br>
- You can then open it in your browser even without being connected to adenine! </div>

---
___

Now we go on with a lecture about the quantification of mapped RNAseq data. 

**=> Lecture 5 : Normalisation and quantification of expression** 

___

<div class="alert alert-block alert-info"> 
    
__*About jupyter notebooks:*__

- To add a new cell, click on the "+" icon in the toolbar above your notebook
- You can "click and drag" to move a cell up or down
- You choose the type of cell in the toolbar above your notebook:
    - 'Code' to enter command lines to be executed 
    - 'Markdown' cells to add text, that can be formatted with some characters
- To execute a 'Code' cell, press SHIFT+ENTER or click on the "play" icon 
- To display a 'Markdown' cell, press SHIFT+ENTER or click on the "play" icon  
- To modify a 'Markdown'cell, double-click on it

  
*To make nice html reports with markdown:* [html visualization tool1](https://dillinger.io/) or [html visualization tool2](https://stackedit.io/app#) and [to draw nice tables](https://www.tablesgenerator.com/markdown_tables ) and the [Ultimate guide](https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd)   
*Further reading on JupyterLab notebooks:*  https://jupyterlab.readthedocs.io/en/latest/user/notebook.html <br>
*Here we are using JupyterLab interface implemented as part of the https://plasmabio.org/ project lead by Sandrine Caburet, Pierre Poulain and Claire Vandiedonck.*

</div>    